# Reprojecting and Resampling Raster Data

This notebook demonstrates how to reproject and resample raster data using `rasterio` in Python. Reprojection changes the coordinate reference system (CRS), while resampling adjusts the pixel resolution. These operations are crucial in remote sensing for aligning datasets.

## Prerequisites
- Install required libraries: `rasterio`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- A sample GeoTIFF file (e.g., `sample.tif`). Replace the file path with your own raster file.

## Learning Objectives
- Reproject a raster to a new CRS.
- Resample a raster to a different resolution.
- Visualize and save the reprojected/resampled raster.

In [ ]:
# Import required libraries
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Load the Original Raster

Load the raster and inspect its current CRS and dimensions.

In [ ]:
# Define the path to the raster file
raster_path = 'sample.tif'

# Open the raster file
with rasterio.open(raster_path) as src:
    original_data = src.read()
    original_crs = src.crs
    original_transform = src.transform
    original_meta = src.meta

# Print original information
print(f'Original CRS: {original_crs}')
print(f'Original Resolution: {original_transform.a, abs(original_transform.e)}')
print(f'Original Shape: {original_data.shape}')

## Step 2: Reproject the Raster

Reproject the raster to a new CRS (e.g., EPSG:4326 - WGS84). Calculate the new transform and dimensions.

In [ ]:
# Define target CRS
target_crs = 'EPSG:4326'

# Calculate transform and dimensions for target CRS
transform, width, height = calculate_default_transform(
    original_crs, target_crs, original_meta['width'], original_meta['height'], *src.bounds
)

# Prepare output metadata
reproject_meta = original_meta.copy()
reproject_meta.update({
    'crs': target_crs,
    'transform': transform,
    'width': width,
    'height': height
})

# Create array for reprojected data
reprojected_data = np.empty((original_data.shape[0], height, width), dtype=original_data.dtype)

# Reproject each band
for i in range(original_data.shape[0]):
    reproject(
        source=original_data[i],
        destination=reprojected_data[i],
        src_transform=original_transform,
        src_crs=original_crs,
        dst_transform=transform,
        dst_crs=target_crs,
        resampling=Resampling.nearest
    )

# Print new information
print(f'Target CRS: {target_crs}')
print(f'Reprojected Shape: {reprojected_data.shape}')

## Step 3: Visualize Reprojected Raster

Visualize the first band of the reprojected raster.

In [ ]:
# Plot the reprojected first band
plt.figure(figsize=(8, 8))
plt.imshow(reprojected_data[0], cmap='gray')
plt.title('Reprojected Raster (First Band)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 4: Resample the Raster

Resample the original raster to a coarser resolution (e.g., double the pixel size).

In [ ]:
# Define scaling factor (e.g., 2 for halving resolution)
scale_factor = 2

# Calculate new dimensions
new_width = original_meta['width'] // scale_factor
new_height = original_meta['height'] // scale_factor

# New transform
new_transform = original_transform * original_transform.scale(
    (original_meta['width'] / new_width),
    (original_meta['height'] / new_height)
)

# Prepare output metadata for resampling
resample_meta = original_meta.copy()
resample_meta.update({
    'transform': new_transform,
    'width': new_width,
    'height': new_height
})

# Create array for resampled data
resampled_data = np.empty((original_data.shape[0], new_height, new_width), dtype=original_data.dtype)

# Resample each band
for i in range(original_data.shape[0]):
    reproject(
        source=original_data[i],
        destination=resampled_data[i],
        src_transform=original_transform,
        src_crs=original_crs,
        dst_transform=new_transform,
        dst_crs=original_crs,
        resampling=Resampling.average  # Use average for downsampling
    )

# Print resampled information
print(f'Resampled Resolution: {new_transform.a, abs(new_transform.e)}')
print(f'Resampled Shape: {resampled_data.shape}')

## Step 5: Visualize Resampled Raster

Visualize the first band of the resampled raster.

In [ ]:
# Plot the resampled first band
plt.figure(figsize=(8, 8))
plt.imshow(resampled_data[0], cmap='gray')
plt.title('Resampled Raster (First Band)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 6: Save Reprojected and Resampled Rasters

Save the reprojected and resampled rasters to new GeoTIFF files.

In [ ]:
# Save reprojected raster
with rasterio.open('reprojected_raster.tif', 'w', **reproject_meta) as dst:
    dst.write(reprojected_data)

# Save resampled raster
with rasterio.open('resampled_raster.tif', 'w', **resample_meta) as dst:
    dst.write(resampled_data)

print('Reprojected raster saved to: reprojected_raster.tif')
print('Resampled raster saved to: resampled_raster.tif')

## Next Steps

- Replace `sample.tif` with your own raster file.
- Experiment with different target CRS (e.g., EPSG:3857 for Web Mercator).
- Try other resampling methods (e.g., Resampling.bilinear, Resampling.cubic).
- Proceed to the next notebook (`07_overlay_vector_on_raster.ipynb`) to overlay vector data on rasters.

## Notes
- Ensure the target CRS is appropriate for your data and analysis.
- Resampling method affects the output quality (e.g., nearest for categorical data, average for continuous).
- See `docs/installation.md` for troubleshooting library installation.